In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import os
import kagglehub
import shutil

In [ ]:
DATASETS = [
    "shivamb/amazon-prime-movies-and-tv-shows",
    "shivamb/disney-movies-and-tv-shows",
    "shivamb/netflix-shows",
]

current_directory = os.getcwd() 
DESTINATION_DIR = os.path.join(current_directory, "data") 

# 1. Cria a pasta 'data' se ela não existir
os.makedirs(DESTINATION_DIR, exist_ok=True)

def download_and_copy_dataset(dataset_name: str, destination_path: str):
    """Baixa um dataset, copia os arquivos para o destino e limpa o cache."""

    # 1. Baixa o dataset para o cache
    try:
        cache_path = kagglehub.dataset_download(dataset_name)
    except Exception as e:
        return

    # 2. Copia os arquivos do cache para o diretório de destino
    for item_name in os.listdir(cache_path):
        source = os.path.join(cache_path, item_name)
        destination = os.path.join(destination_path, item_name)

        # Copia apenas arquivos (ignorando subpastas)
        if os.path.isfile(source):
            shutil.copy2(source, destination) 

    # 3. Remove completamente a pasta do cache
    try:
        shutil.rmtree(cache_path)
    except OSError as e:
        print(f"  > AVISO: Não foi possível remover o cache: {e}")
        
for dataset in DATASETS:
    download_and_copy_dataset(dataset, DESTINATION_DIR)

print("\n\n--- Processo Finalizado ---")
print(f"Todos os arquivos dos datasets estão na pasta: {DESTINATION_DIR}")

In [ ]:
#Concatenar as base de dados 

DESTINATION_DIR = os.path.join(current_directory, "data") 

df_netflix = pd.read_csv(os.path.join(DESTINATION_DIR, 'netflix_titles.csv'))
df_disney = pd.read_csv(os.path.join(DESTINATION_DIR, 'disney_plus_titles.csv'))
df_amazon = pd.read_csv(os.path.join(DESTINATION_DIR, 'amazon_prime_titles.csv'))

df_netflix['streaming'] = 'Netflix'
df_disney['streaming'] = 'Disney+'
df_amazon['streaming'] = 'Prime Video'

dataframes_to_concat = [df_netflix, df_disney, df_amazon]

df_streaming = pd.concat(dataframes_to_concat, ignore_index=True)

df_streaming['date_added'] = df_streaming['date_added'].str.strip()

df_streaming['date_added'] = pd.to_datetime(df_streaming['date_added'], format='%B %d, %Y')

# Exibindo informações gerais para confirmar a junção e os tipos de dados
print("\nInformações do DataFrame consolidado:")
df_streaming.head(5)